# Spanning trees

Anton Antonov   
July 2024

----

## Setup

In [1]:
use Graph;

use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Star;
use Graph::Path;
use Graph::Petersen;
use Graph::Wheel;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;
use Math::DistanceFunctions;
use Math::Nearest;
use Hash::Merge;

use JavaScript::D3;
use WWW::MermaidInk;

In [2]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [3]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

In [4]:
my $title-color = 'Silver';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

{background => 1F1F1F, edge-thickness => 3, title-color => Silver, vertex-size => 6}

-----

## Small Geo-points set

A company is planning a fiber network for a number of Chicago suburbs. It only has the right of way for its fiber along certain corridors. Some of those corridors might be more expensive. Find the subgraph of connection corridors that connect every suburb with the lowest total cost:

In [5]:
my @dsEdges =
        %('from'=>'Maywood','to'=>'Downers','weight'=>14),
        %('from'=>'Maywood','to'=>'Oakbrook','weight'=>28),
        %('from'=>'Maywood','to'=>'Addison','weight'=>10),
        %('from'=>'Downers','to'=>'Grove','weight'=>25),
        %('from'=>'Downers','to'=>'Stickney','weight'=>23),
        %('from'=>'Downers','to'=>'Wheaton','weight'=>20),
        %('from'=>'Downers','to'=>'Oakbrook','weight'=>27),
        %('from'=>'Oakbrook','to'=>'Stickney','weight'=>11),
        %('from'=>'Oakbrook','to'=>'Addison','weight'=>17),
        %('from'=>'Addison','to'=>'McCook','weight'=>21),
        %('from'=>'Grove','to'=>'Wheaton','weight'=>10),
        %('from'=>'Stickney','to'=>'Wheaton','weight'=>15),
        %('from'=>'Stickney','to'=>'Bellwood','weight'=>20),
        %('from'=>'Wheaton','to'=>'Bellwood','weight'=>10),
        %('from'=>'Bellwood','to'=>'McCook','weight'=>28);

deduce-type(@dsEdges)

Vector(Struct([from, to, weight], [Str, Str, Int]), 15)

In [6]:
my $g = Graph.new(@dsEdges)

Graph(vertexes => 9, edges => 15, directed => False)

In [7]:
#% js
$g.edges(:dataset)
==> js-d3-graph-plot(
    width => 800,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => { strength => -800}, collision => {radius => 50}}
    )

In [8]:
my $stree = $g.find-spanning-tree;

Graph(vertexes => 9, edges => 8, directed => False)

In [9]:
#% js
$g.edges
==> js-d3-graph-plot(
    highlight => $stree.edges,
    width => 800,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => { strength => -800}, collision => {radius => 50}}
    )

-------

## Grid graph

In [26]:
my $g = Graph::Grid.new(n => 4, m => 6)

Graph(vertexes => 24, edges => 38, directed => False)

In [27]:
my $stree = $g.find-spanning-tree

Graph(vertexes => 24, edges => 23, directed => False)

In [30]:
#% js
$g.edges
==> js-d3-graph-plot(
    highlight => [|$stree.vertex-list, |$stree.edges],
    width => 500,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => {strength => -160, iterations => 2}, collision => {radius => 4, iterations => 1}, link => {distance => 0}}
    )

-----

## Florida cities

Build an interstate highway system joining the cities of the state of Florida, USA.

Here is a dataset of large enough cities:

In [59]:
my @cities = city-data().grep({ $_<State> eq 'Florida' && $_<Population> ≥ 50_000 });
my @cities = city-data().grep({ $_<Country> eq 'Bulgaria' && $_<Population> ≥ 10_000 });
@cities.elems

84

In [60]:
#% html
@cities.head(4) ==> to-html

ID,Country,City,State,Elevation,Population,Longitude,Latitude,LocationLink
Bulgaria.Sofia_grad.Sofia,Bulgaria,Sofia,Sofia grad,570,1260120,23.31,42.69,"http://maps.google.com/maps?q=42.69,23.31&z=12&t=h"
Bulgaria.Plovdiv.Plovdiv,Bulgaria,Plovdiv,Plovdiv,170,346757,24.75,42.15,"http://maps.google.com/maps?q=42.15,24.75&z=12&t=h"
Bulgaria.Varna.Varna,Bulgaria,Varna,Varna,16,335177,27.900000000000002,43.21,"http://maps.google.com/maps?q=43.21,27.900000000000002&z=12&t=h"
Bulgaria.Burgas.Burgas,Bulgaria,Burgas,Burgas,30,206371,27.47,42.51,"http://maps.google.com/maps?q=42.51,27.47&z=12&t=h"


Here is the corresponding weighted graph:

In [61]:
my %coords = @cities.map({ $_<City> => ($_<Latitude>, $_<Longitude>) });
my @dsEdges = (%coords.keys X %coords.keys ).map({ %(from => $_.head, to => $_.tail, weight => geo-distance(|%coords{$_.head}, |%coords{$_.tail} )) })

[{from => Sandanski, to => Sandanski, weight => 0} {from => Sandanski, to => Varna, weight => 421128.02915672085} {from => Sandanski, to => Kostinbrod, weight => 140381.3835654189} {from => Sandanski, to => Haskovo, weight => 193206.29735361272} {from => Sandanski, to => Sevlievo, weight => 221326.62179573855} {from => Sandanski, to => Rakovski, weight => 161131.41784987602} {from => Sandanski, to => Dolni Dabnik, weight => 225534.41756830024} {from => Sandanski, to => Smoljan, weight => 117466.4357687397} {from => Sandanski, to => Trojan, weight => 190110.64981253937} {from => Sandanski, to => Velingrad, weight => 78799.96893041863} {from => Sandanski, to => Glavinica, weight => 391784.4341425147} {from => Sandanski, to => Gorna Orjahovica, weight => 264294.2110722536} {from => Sandanski, to => Goce Delcev, weight => 37543.48225932714} {from => Sandanski, to => Blagoevgrad, weight => 54889.951411085836} {from => Sandanski, to => Provadija, weight => 386687.0467047009} {from => Sandans

In [62]:
my $g = Graph.new(@dsEdges, :!directed)

Graph(vertexes => 84, edges => 3528, directed => False)

In [63]:
my $stree = $g.find-spanning-tree

Graph(vertexes => 84, edges => 83, directed => False)

In [64]:
#% js
$stree.edges
==> js-d3-graph-plot(
    width => 1500,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => {strength => -160, iterations => 2}, collision => {radius => 4, iterations => 1}, link => {distance => 0}}
    )